In [54]:
from open3d import utility
from open3d import geometry
from open3d import io
from open3d import visualization
from open3d import registration
import numpy as np
import copy
import time
import os
import math
import glob
import random
from ipywidgets import FloatProgress
from IPython.display import display

In [55]:
def read_mesh(filedir):
    assert os.path.exists(filedir), 'File Directory does not exist'
    assert filedir.endswith('.ply'), 'Mesh File must be OBJ or PLY'
    print('PLY Mesh File Loaded')
    mesh = io.read_triangle_mesh(filedir)
    return mesh

def read_pcd(filedir):
    assert os.path.exists(filedir), 'File Directory does not exist'
    assert filedir.endswith('.ply'), 'Mesh File must be OBJ or PLY'
    print('PLY File Loaded')
    pcd = io.read_point_cloud(filedir)
    return pcd

def create_mesh_line_set(mesh, color=[1,0,0]):
    line_set = geometry.create_line_set_from_triangle_mesh(mesh)
    colors = [color for _ in range(len(line_set.lines))]
    line_set.colors = utility.Vector3dVector(colors)
    return line_set

def create_mesh_pcd(mesh):
    pcd = geometry.PointCloud()
    pcd.points = utility.Vector3dVector(mesh.vertices)
    mesh.compute_vertex_normals()
    pcd.normals = utility.Vector3dVector(mesh.vertex_normals)
    return pcd

def create_pcd(points):
    pcd = geometry.PointCloud()
    pcd.points = utility.Vector3dVector(np.asarray(points))
    geometry.estimate_normals(pcd, search_param=geometry.KDTreeSearchParamKNN(10))
    return pcd

In [56]:
def extract_pom(base_mesh, pomdir, transformation, radius=5.0, line_set=False):
    pom_mesh = read_mesh(pomdir)
    pom_mesh.transform(transformation)
    halfmesh = geometry.create_half_edge_mesh_from_mesh(pom_mesh)
    mesh_vertices = np.asarray(halfmesh.vertices)
    edge_vertices = set()
    for edge in halfmesh.half_edges:
        if edge.twin == -1:
            edge_vertices.update(set(edge.vertex_indices))
            
    open_vertices = []
    for _, idx in enumerate(edge_vertices):
        open_vertices.append(mesh_vertices[idx])

    tree = geometry.KDTreeFlann(create_mesh_pcd(base_mesh))
    base_vertices = np.asarray(base_mesh.vertices)
    closest_vertices = []
    for vertex in open_vertices:
        [_, idx, _] = tree.search_knn_vector_3d(vertex, 1)
        closest_vertices.append(base_vertices[np.asarray(idx)[0]])

    mean_point = np.asarray(closest_vertices).mean(axis=0)
    [_, idx, _] = tree.search_knn_vector_3d(mean_point, 1)
    shoulder_point = base_vertices[np.asarray(idx)[0]]
    closest_vertices.append(shoulder_point)
    pom_marker = geometry.create_mesh_sphere(radius=radius)
    pom_marker.compute_vertex_normals()
    pom_marker.paint_uniform_color([0.1, 0.5, 1])
    pom_marker.vertices = utility.Vector3dVector(np.asarray(pom_marker.vertices) + 
                                                 shoulder_point)
    pcd_closest = create_pcd(closest_vertices)
    pcd_edge = create_pcd(open_vertices)
    if line_set:
        pom_line_set = geometry.create_line_set_from_triangle_mesh(pom_mesh)
        return [pom_line_set, pcd_closest, pcd_edge, pom_marker]
    return [pom_mesh, pcd_closest, pcd_edge, pom_marker]

In [57]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    visualization.draw_geometries([source_temp, target_temp])
    
def align_mesh(meshdir, markermeshdir, max_correspondence=100, max_iter=10000):
    source_mesh = read_mesh(markermeshdir)
    target_mesh = read_mesh(meshdir)
    source = create_mesh_pcd(source_mesh)
    target = create_mesh_pcd(target_mesh)
    result = registration.registration_icp(
        source, target,
        max_correspondence_distance=max_correspondence,
        estimation_method=registration.TransformationEstimationPointToPlane(),
        criteria=registration.ICPConvergenceCriteria(max_iteration=max_iter))
    print(f'Fitness: {result.fitness}')
    print(f'Inlier RMSE: {result.inlier_rmse}')
    return target_mesh, source_mesh.transform(result.transformation), result.transformation

In [14]:
folderdir = '/Users/nickf/Dropbox/Nicky T-Shirt Project/Subject Scans/Male Scans/Nicholas Ferrara/NF_SCAN_V2/'
meshdir = f'{folderdir}NF_V2_M2.ply'
markermeshdir = f'{folderdir}NFMARKERS_V2_M2.ply'
base_mesh, aligned_marker, transformation = align_mesh(meshdir, markermeshdir,
                                                       max_correspondence=1000,
                                                       max_iter=100)

PLY Mesh File Loaded
PLY Mesh File Loaded
Fitness: 1.0
Inlier RMSE: 8.632307212722644


In [15]:
visualization.draw_geometries([geometry.create_line_set_from_triangle_mesh(base_mesh), 
                               aligned_marker])

In [16]:
pom_geometries = []
pom_markers = []
pomdir = f'{folderdir}NF_V2_M2_ShoulderLeft.ply'
shoulderLeft  = extract_pom(base_mesh, pomdir, transformation, radius=15.0, line_set=True)
pom_geometries.extend(shoulderLeft)
pom_markers.append(shoulderLeft[-1])
pomdir = f'{folderdir}NF_V2_M2_ShoulderRight.ply'
shoulderRight = extract_pom(base_mesh, pomdir, transformation, radius=15.0, line_set=True)
pom_geometries.extend(shoulderRight)
pom_markers.append(shoulderRight[-1])

PLY Mesh File Loaded
PLY Mesh File Loaded


In [17]:
geometries = [base_mesh]
geometries.extend(pom_geometries)
geometries.extend(pom_markers)
visualization.draw_geometries(geometries)